# Reverse Parking - Dynamic Obstacles

## Import Libraries

In [13]:
# Install environment and visualization dependencies
!pip install highway-env

In [14]:
import gymnasium as gym
import highway_env
from matplotlib import pyplot as plt
from IPython.display import HTML
import imageio

%matplotlib inline

## Static Environment

In [15]:
env = gym.make(
    'parking-v0',
    render_mode='rgb_array',

    config={
    "observation": {
        "type": "KinematicsGoal",
        "features": ['x', 'y', 'vx', 'vy', 'cos_h', 'sin_h'],
        "scales": [100, 100, 5, 5, 1, 1],
        "normalize": False,
    },
    "action": {
        "type": "ContinuousAction"
    },
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "screen_width": 600,
    "screen_height": 300,
    "centering_position": [0.5, 0.5],
    "scaling": 7,
    "show_trajectories": False,
    "render_agent": True,
    "offscreen_rendering": False,
    "vehicles_count": 5
})

env.reset()

# ---- Run one episode and collect frames ----
frames = []
obs, info = env.reset()
done = False

for step in range(150):  # run for 150 steps
    action = env.action_space.sample()  # random driving
    obs, reward, done, truncated, info = env.step(action)
    frame = env.render()  # get the RGB frame
    frames.append(frame)
    if done:
        break

env.close()

# ---- Save as a video file ----
output_path = "ego_parking_demo.mp4"
imageio.mimsave(output_path, frames, fps=15)
print(f"🎥 Video saved as {output_path}")

# ---- Display video inside notebook (if using Jupyter/Colab) ----
from IPython.display import Video
Video(output_path, embed=True)

🎥 Video saved as ego_parking_demo.mp4


## Dynamic Environment

In [ ]:
import gymnasium as gym
import highway_env
import numpy as np
import imageio
from highway_env.vehicle.behavior import IDMVehicle
from IPython.display import Video

# ---- Create parking environment ----
env = gym.make("parking-v0", render_mode="rgb_array",
               config={
    "observation": {
        "type": "KinematicsGoal",
        "features": ['x', 'y', 'vx', 'vy', 'cos_h', 'sin_h'],
        "scales": [100, 100, 5, 5, 1, 1],
        "normalize": False,
    },
    "action": {
        "type": "ContinuousAction"
    },
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "screen_width": 600,
    "screen_height": 300,
    "centering_position": [0.5, 0.5],
    "scaling": 7,
    "show_trajectories": False,
    "render_agent": True,
    "offscreen_rendering": False,
})

# ---- Add moving IDM vehicles after collision ----
def add_idm_vehicles(env, n=3):
    base_env = env.unwrapped

    # Remove everything except ego
    base_env.road.vehicles = [base_env.vehicle]

    for _ in range(n):
        v = IDMVehicle.create_random(base_env.road)
        v.position = np.random.uniform(-20, 20, size=2)
        v.heading = 0

        # Slow speeds
        v.speed = np.random.uniform(1.0, 3.0)
        v.target_speed = np.random.uniform(2.0, 4.0)

        v.max_acc = 0.5
        v.max_decel = 1.0

        base_env.road.vehicles.append(v)


base_env = env.unwrapped
obs, info = env.reset()
add_idm_vehicles(env, n=3)

# Keep only ego vehicle
base_env.road.vehicles = [base_env.vehicle]

# ---- Add moving IDM vehicles ----
for _ in range(3):
    v = IDMVehicle.create_random(base_env.road)
    v.position = np.random.uniform(-20, 20, size=2)
    v.heading = 0  # face along x-axis

    v.speed = np.random.uniform(1.0, 3.0)    # Slow 1–3 m/s

    # Also reduce acceleration aggressiveness
    v.target_speed = np.random.uniform(2.0, 4.0)
    v.max_acc = 0.5
    v.max_decel = 1.0

    base_env.road.vehicles.append(v)

# ---- Run simulation ----
frames = []
for step in range(150):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    base_env.road.act()   # update motion of IDM vehicles
    frames.append(env.render())
    if done:
        obs, info = env.reset()
        add_idm_vehicles(env, n=3)

env.close()

# ---- Save as video ----
output_path = "reverse_parking_dynamic_cars.mp4"
imageio.mimsave(output_path, frames, fps=15)
print(f"🎥 Video saved as {output_path}")
Video(output_path, embed=True)